# get_gdps

This routine grabs the latest Global Deterministic Prediction System (GDPS) model data from MSC Datamart and outputs png plots into designated product folders for each ACMWF product.

**Datamart:**

https://eccc-msc.github.io/open-data/msc-data/nwp_gdps/readme_gdps-datamart_en/#levels

https://dd.weather.gc.ca/model_gem_global/

**Outputs:**

Mountain Weather Forecast -> *500 mbar, Surface* -> ```"mwf500"```, ```"mwfsurf"```

Temperatures -> *1500m 0400, 1500m 1600* -> ```"temp1504"```, ```"temp1516"```

Surface Maps -> *0-144 Hours* -> ```"surf"```

500 mbar & Precipitable Water -> *500 mbar, Precipitable Water* ```"mbar500"```, ```"pwat"```

**Maintenance:**

For bug reports, suggestions, inquiries, contact Andrew.Loeppky@gmail.com

In [0]:
from herbie import Herbie
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd

from matplotlib.patches import Rectangle
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pytz
import os, shutil
import json

FileNotFoundError: [Errno 2] No usable temporary directory found in ['/var/folders/gw/d8k1vszn7ws41z7c0209d9500000gn/T/', '/tmp', '/var/tmp', '/usr/tmp', '/Users/andrew/Repos/BMWF/code']

In [ ]:
# time of last model run (0Z and 12Z)
now = pd.Timestamp.utcnow().floor("12h").tz_localize(None)

# config determines the directories in which to save each graphic
with open("..//config/gdps_config.json") as f:
    config = json.load(f)

# 10 day run (240 hours) with 3 hr spacing
tstep = range(3,243,3)

In [ ]:
# clear the plot folder
for directory in config["plots"].values():
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


In [ ]:
for i in tstep:
    # initialize figures
    mwf500, ax0 = plt.subplots()
    surf, ax1 = plt.subplots()
    
    # populate figures
    ax0.text(0.5, 0.5, f"test {i}")
    ax1.text(0.5, 0.5, f"test {i}")
    
    # save figures
    mwf500.savefig(f"{config["plots"]["mwf500"]}mwf500_{i}")
    plt.close(fig=mwf500)
    
    surf.savefig(f"{config["plots"]["mwfsurf"]}mwfsurf{i}")
    plt.close(fig=surf)
    
    